In [ ]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from transformers import (AutoModel,AutoModelForMaskedLM, 
                          AutoTokenizer, LineByLineTextDataset,
                          DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments)

「Transformer」は、過去に自然言語処理分野で多く使われていた「RNN」（Recurrent Neural Network）や「CNN」（Convolutional Neural Network）を「Self-Attention Layer」に入れ替えたモデルで、発表以来、多くの自然言語処理モデルが「Transformer」で再構築され、過去を上回る結果を上げました。

In [ ]:
train_data = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test_data = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

data = pd.concat([train_data,test_data])

#data['excerpt']に対して改行をなくす。
data['excerpt'] = data['excerpt'].apply(lambda x: x.replace('\n',''))

#改行をなくしたexcerptを改行で全結合、textに保存.
text  = '\n'.join(data.excerpt.tolist())

with open('text.txt','w') as f:
    f.write(text)

In [ ]:
#pretrainモデルであるroberta-baseをmodelとして指定.
model_name = 'roberta-large'
model = AutoModelForMaskedLM.from_pretrained(model_name)

#Tokenizer
#Tokenizerは保存しておく(なんのために？)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained('./clrp_roberta_large');

In [ ]:
#この段階ではtrain/validともに同じ全結合されたデータが元になったdataset状態。
#tokenizerを指定して、file_pathの文字列を区切ったものになっている？
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="text.txt", #mention train text file here
    block_size=256)

valid_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="text.txt", #mention valid text file here
    block_size=256)

#data_collator 
#Data collators are objects that will form a batch by using a list of 
#dataset elements as input. 
#These elements are of the same type as the elements of train_dataset or eval_dataset.
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

#training_argsはtrainerで指定するハイパーパラメータのことで、TrainingArguments関数で一式を必要なフォーマットにて与える
training_args = TrainingArguments(
    output_dir="./clrp_roberta_large_chk", #select model path for checkpoint
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy= 'steps',
    save_total_limit=2,
    eval_steps=200,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    load_best_model_at_end =True,
    prediction_loss_only=True,
    report_to = "none")

#プリトレイニングのクラスを作成。
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
#今回のお題になっているexcerptをroberta_baseモデルにプレ学習させて、
#clrp_robertaモデルとして保存している.
#以下の１行を動作させようとするとメモリが15GBit超えてSTOPする.
trainer.train()

In [ ]:

trainer.save_model(f'./clrp_roberta_large')